# Final Dashboard: Anomaly Detection Simulation
**Student Name:** Ali Cihan Ozdemir  
**Course:** CSCN8010 Practical Lab 1

## Objectives
1. Load Synthetic Test Data (with injected anomalies).
2. Run the `AlertSystem` on the data stream.
3. Visualize the signal, regression line, and detected Alert/Error events.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add src to path
sys.path.append('../src')
from alert_system import AlertSystem

# Load Data
test_data_path = '../data/test_data_synthetic.csv'
if not os.path.exists(test_data_path):
    print("Test data not found! Please run src/generate_test_data.py")
else:
    df = pd.read_csv(test_data_path)
    print("Loaded test data.")
    df.head()

## 2. Simulate Stream & Detect Anomalies
We will iterate through the data and use our `AlertSystem` to detect anomalies in real-time (simulation).
We enable `verbose` mode for a short segment to demonstrate the event logging logic.

In [ ]:
# Initialize System
alert_system = AlertSystem()

# Run Simulation
print("Running full simulation...")
# Run fully to capture all history for plotting
alert_system.run_simulation(df.to_dict('records'), verbose=False)

print("\n--- Event Decision Trace (Sample of Logic) ---")
print("Demonstrating how incoming data triggers alerts (Past History + Present Reading)...")
# Re-run a small slice with verbose=True to demonstrate the "decision per row" capability
sample_slice = df.iloc[48:60] # Slice around the first spike
alert_system.active_violations = {} # Reset state for demo
for index, row in sample_slice.iterrows():
    alert_system.check_stream(row, verbose=True)

print(f"\nSimulation Check: Detected {len(alert_system.events)} events.")

## 3. Visualization
Plotting the test data with regression lines and markers for detected events using the `plot_results` method.

In [ ]:
alert_system.plot_results()